# This notebook is to take apart video feed and turn each frame into an image that then can be run in MegaDetector.

In [6]:
!pip install opencv-python-headless 

# Keep this just incase an issue happens.


In [7]:
import os
import cv2
import json
from megadetector.utils import path_utils
from megadetector.detection import video_utils
from megadetector.utils.ct_utils import write_json
from megadetector.visualization import visualization_utils as vis_utils
from megadetector.detection.video_utils import frame_results_to_video_results, FrameToVideoOptions

# Add the directory containing video_utils.py to the system path
import sys
sys.path.append("C:/Users/Public/Documents/MegaDetector_App/MegaDetector/megadetector/detection")

from video_utils import video_folder_to_frames, frames_to_video, get_video_fs, frame_results_to_video_results
import tkinter as tk
from tkinter import filedialog as fd 

# Create a hidden root window
root = tk.Tk()
root.withdraw()
root.attributes('-topmost', True)  # Ensure it stays on top
# Configure paths
# input_folder = r"C:/Users/Public/Documents/Data/video_data/"
# output_folder_base = r"C:/Users/Public/Documents/Data/video_data/frames/"
# detected_frame_folder_base = r"C:/Users/Public/Documents/Data/video_data_detected/"
# rendered_videos_folder_base = r"C:/Users/Public/Documents/Data/video_data_rendered/"
# json_path = r"C:/Users/Public/Documents/jsons/tests.json"




''

In [8]:
#The following 5 cells each run the same function, Ask directory, they are split up into seperate cells to avoid confusion when choosing folders
# the Ask open file name function allows you to manually select the folder that you are using each time.
#the print function will print the folder name, if it isnt the desired folder simply run the cell again the chose a different one

In [9]:
input_folder = fd.askdirectory()
print("Selected folder:", input_folder)

Selected folder: C:/Users/Public/Public_Documents/jsons


In [10]:
output_folder_base = fd.askdirectory() # change name to frame folder
print("Selected folder:",output_folder_base)

Selected folder: C:/Users/Public/Public_Documents/jsons


In [11]:
detected_frame_folder_base = fd.askdirectory() # detections folder change name
print("Selected folder:", detected_frame_folder_base)

Selected folder: C:/Users/Public/Public_Documents/jsons


In [12]:
rendered_videos_folder_base = fd.askdirectory() # rename 
print("Selected folder:", rendered_videos_folder_base)

Selected folder: C:/Users/Public/Public_Documents/jsons


In [13]:
# Ask the user to select OR create a JSON file
json_path = fd.asksaveasfilename()
if not json_path:
    raise ValueError("No JSON file selected")
# If the file doesn’t exist, create an empty one so MegaDetector is happy
if not os.path.exists(json_path):
    with open(json_path, "w") as f:
        f.write("{}")  # valid empty JSON

# json_path = fd.askopenfilename()
# print("Selected folder:", json_path)

In [14]:
# Ensure output directories exist
os.makedirs(detected_frame_folder_base, exist_ok=True)
os.makedirs(rendered_videos_folder_base, exist_ok=True)

In [15]:
#From Don Morris Mega detector examples,Managing a local mega detector video batch Notebook
quality = 90 #controls image output quality, lower number smaller files higher number clearer images (out of 100)
max_width = 1600 #controls Frame size, resizes to a max width, avoids huge image files
recursive = True #looks through sub folders,if changes to false only the outer most folders will be processed (must be true to do folders within folders)
overwrite = True #Replaces existing frames (duplicates), if changed to false and duplicates exist extraction might fail or skip (DO NOT TOUCH)
parallelization_uses_threads = True #Speeds up the processing
n_workers = 8 #Controls how many threads are used

# Sample every Nth frame.  To specify a sampling rate in seconds, use a negative
# value.  For example:
#
# * Setting every_n_frames to -2.0 yields a frame rate of 0.5 fps
# * Setting every_n_frames to -0.5 yields a frame rate of 2.0 fps
#
every_n_frames = 10 #Controls how often a frame is extracted, (every 10 frames), without this too few or too many frames may be extracted (DO NOT DELETE)

In [16]:
# Split videos into frames
# Ensure folders exist
assert os.path.isdir(input_folder)
os.makedirs(output_folder_base, exist_ok=True)

# Frame extraction
frame_filenames_by_video, fs_by_video, video_filenames = \
    video_folder_to_frames(
        input_folder=input_folder,
        output_folder_base=output_folder_base,
        recursive=recursive,
        overwrite=overwrite,
        n_threads=n_workers,
        every_n_frames=every_n_frames,
        parallelization_uses_threads=parallelization_uses_threads,
        quality=quality,
        max_width=max_width,
        #allow_empty_videos=True
    )

Found 0 videos in folder C:/Users/Public/Public_Documents/jsons


In [17]:
frame_folder_base = output_folder_base

In [18]:
# Each leaf-node folder *should* correspond to a video; we're going to verify that below.

from collections import defaultdict

frame_files = path_utils.find_images(frame_folder_base,recursive=True)
frame_files = [s.replace('\\','/') for s in frame_files]
print('Enumerated {} total frames'.format(len(frame_files)))

# Find unique (relative) folders
folder_to_frame_files = defaultdict(list)

# fn = frame_files[0]
for fn in frame_files:
    folder_name = os.path.dirname(fn)
    folder_name = os.path.relpath(folder_name,frame_folder_base)
    folder_to_frame_files[folder_name].append(fn)

print('Found {} folders for {} files'.format(len(folder_to_frame_files),len(frame_files)))

Enumerated 0 total frames
Found 0 folders for 0 files


In [19]:
video_filenames = video_utils.find_videos(input_folder,recursive=True)
video_filenames = [os.path.relpath(fn,input_folder) for fn in video_filenames]
video_filenames = [fn.replace('\\','/') for fn in video_filenames]
print('Input folder contains {} videos'.format(len(video_filenames)))

Input folder contains 0 videos


In [20]:
# 1. Run MegaDetector on frames (shell command)
!python "C:/Users/Public/Documents/MegaDetector_App/MegaDetector/megadetector/detection/run_detector_batch.py" \
MDV5A {output_folder_base} {json_path} \
    --output_relative_filenames \
    --recursive \
    --checkpoint_frequency 1000 \
    --quiet

Bypassing download of already-downloaded file md_v5a.0.0.pt
No image files found in directory C:/Users/Public/Public_Documents/jsons, exiting


In [ ]:
# Run this if you want bounding boxes drawn on images
!python "C:/Users/Public/Documents/MegaDetector_App/MegaDetector/megadetector/postprocessing/postprocess_batch_results.py" \
    "{json_path}" "{detected_frame_folder_base}" --image_base_dir "{output_folder_base}" --num_images_to_sample -1


In [ ]:
# Verify paths and files
print(os.path.isfile(json_path))  # Should print True if the file exists
print(os.path.isdir(output_folder_base))  # Should print True if the directory exists
print(os.path.isdir(detected_frame_folder_base))  # Should print True if the directory exists


In [ ]:
# Process each video folder separately to create individual output videos

# IMPORTANT: Update these paths before running!
detected_frames_folder = detected_frame_folder_base + "/detections"  # Where postprocess_batch_results.py saved the frames
rendered_videos_output_folder = rendered_videos_folder_base  # Where you want final videos saved

# Ensure output directory exists
os.makedirs(rendered_videos_output_folder, exist_ok=True)

# Get frame rate from one of the original videos
# Find the first video in your input folder to get frame rate
sample_videos = video_utils.find_videos(input_folder, recursive=True)
if not sample_videos:
    raise ValueError("No videos found in input folder to determine frame rate")
sample_video_file = sample_videos[0]
print(f"Using frame rate from: {sample_video_file}")
Fs = get_video_fs(sample_video_file)

# Get all subdirectories in the detections folder (each represents a video)
video_folders = [f for f in os.listdir(detected_frames_folder) 
                 if os.path.isdir(os.path.join(detected_frames_folder, f))]

print(f"Found {len(video_folders)} video folders to process")

# Process each video folder separately
for video_folder_name in video_folders:
    video_folder_path = os.path.join(detected_frames_folder, video_folder_name)
    
    # Collect all frame files from this specific video folder
    frame_files = sorted([os.path.join(video_folder_path, f) 
                         for f in os.listdir(video_folder_path) 
                         if f.endswith(('.jpg', '.png'))])
    
    if not frame_files:
        print(f"No frames found in {video_folder_name}, skipping...")
        continue
    
    # Create output video filename based on the folder name
    output_video_filename = os.path.join(rendered_videos_output_folder, f"{video_folder_name}.mp4")
    
    # Determine the width and height from the first image
    frame = cv2.imread(frame_files[0])
    if frame is None:
        print(f"Could not read first frame in {video_folder_name}, skipping...")
        continue
    
    height, width, channels = frame.shape
    
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 files
    out = cv2.VideoWriter(output_video_filename, fourcc, Fs, (width, height))
    
    # Write all frames to the video
    for image_path in frame_files:
        frame = cv2.imread(image_path)
        if frame is not None:
            out.write(frame)
    
    out.release()
    print(f"✅ Created video: {output_video_filename} ({len(frame_files)} frames)")

print(f"\n🎉 All videos processed! Output saved to: {rendered_videos_output_folder}")